In [21]:
import os,json
import random
import shutil
from tqdm import tqdm
split_dataset_flag = True # 运行之前先确认下是否真的需要划分数据集

In [22]:
# random.seed(101001)
random.random()

0.7617361127467278

## 将数据集 划分为 训练集/验证集

In [23]:
# 该代码段不一定要之执行
tot_corpus = r'det_bookspine_text/new_label_samples'

# ---------- 为训练集准备数据 --------
train_img_dir = r'det_bookspine_text/train'
if os.path.exists(train_img_dir):
    shutil.rmtree(train_img_dir)
os.makedirs(train_img_dir)

# ---------- 为验证集准备数据 --------
val_img_dir = r'det_bookspine_text/val'
if os.path.exists(val_img_dir):
    shutil.rmtree(val_img_dir)
os.makedirs(val_img_dir)


train_ratio = 0.85
if split_dataset_flag:
    for i in tqdm(os.listdir(tot_corpus)):
        if i.endswith('.jpg'):
            img_path = os.path.join(tot_corpus, i)
            json_path = os.path.join(tot_corpus, i.replace('.jpg', '.json'))
            if random.random() < train_ratio:
                # 这里面都为训练数据
                dst_img_path = os.path.join(train_img_dir, i)
            else: # 这里面都为验证数据
                dst_img_path = os.path.join(val_img_dir, i)
            shutil.copy(img_path, dst_img_path)

100%|██████████| 326/326 [00:00<00:00, 9884.00it/s]


# paddleOCR-det 数据格式要求 https://github.com/PaddlePaddle/PaddleOCR/blob/main/doc/doc_ch/dataset/ocr_datasets.md
" 图像文件名        中间用"\t"分隔      json.dumps编码的图像标注信息"
ch4_test_images/img_61.jpg    [{"transcription": "MASA", "points": [[310, 104], [416, 141], [418, 216], [312, 179]]}, {...}]

## 读取标签文件（.json） 中的内容

In [24]:
def get_corrd_from_json(json_path):
    with open(json_path, 'r', encoding='utf-8') as f:
        json_data = json.load(f)['shapes']
    print(f"json_data:{json_data}")
    # contous_list = []    
    boxes_info_list = [] 
    for zone_info in json_data:
        # print(f"zone_info:{zone_info}")
        points = zone_info["points"]
        classes = zone_info["label"]
        # points = [tuple(point) for point in points]
        boxes_info_list.append({"box":points, "class":classes})
    return boxes_info_list

In [25]:
# test/1123.jpg	 [ 
#                 {"transcription": 0, "points": [[202, 129], [221, 124], [241, 120], [261, 116], [280, 111], [300, 107], [320, 103], [319, 83], [299, 87], [279, 92], [259, 97], [239, 101], [219, 106], [200, 111]]}, 
#                  {"transcription": 0, "points": [[183, 174], [210, 167], [238, 161], [266, 155], [294, 149], [322, 143], [350, 137], [347, 105], [318, 111], [289, 117], [260, 124], [231, 130], [202, 136], [174, 143]]}, 
#                  {"transcription": 0, "points": [[208, 202], [228, 199], [249, 196], [269, 193], [290, 190], [310, 187], [331, 185], [328, 144], [306, 149], [284, 154], [262, 159], [240, 164], [218, 169], [196, 175]]}, 
#                  {"transcription": 0, "points": [[1, 319], [9, 318], [18, 317], [27, 316], [36, 315], [45, 314], [54, 313], [47, 275], [39, 276], [31, 278], [24, 279], [16, 281], [8, 282], [1, 284]]}, 
#                  {"transcription": 0, "points": [[9, 333], [27, 332], [46, 332], [65, 332], [83, 331], [102, 331], [121, 331], [117, 307], [97, 310], [78, 314], [59, 318], [39, 321], [20, 325], [1, 329]]}, 
#                  {"transcription": 0, "points": [[474, 273], [477, 272], [481, 272], [485, 272], [489, 272], [493, 272], [497, 272], [497, 236], [491, 237], [485, 238], [479, 239], [473, 240], [467, 241], [461, 242]]}, 
#                  {"transcription": 0, "points": [[216, 248], [228, 235], [244, 227], [259, 220], [275, 214], [292, 210], [310, 209], [311, 185], [291, 188], [272, 194], [253, 199], [236, 210], [219, 219], [203, 230]]}
#                 ]


In [26]:
def make_label(tot_corpus = r"det_bookspine_text/new_label_samples",
               img_dir = r'det_bookspine_text/train',
               datatype = 'train' # 'test'
               ):
    
    
    with open('det_bookspine_text/%s.txt'%datatype, 'w', encoding='utf-8') as write:
        for i in os.listdir(img_dir):
            if i.endswith('.jpg'):
                json_path = os.path.join(tot_corpus, i.replace('.jpg','.json'))
                contous_list = get_corrd_from_json(json_path) # contous_list = [ {}]
                print(f'从json标签文件中获取的内容：{contous_list}')
                boxes =[ [ [int(x), int(y) ] for x, y in contous["box"] ] for contous in  contous_list]
                classes = [contous["class"] for contous in  contous_list]
                this_img_det_rec = []
                for box_, class_ in zip(boxes, classes):
                    # contous 是一个文本框
                    # [[x0,y0],[x1,y1],[x2,y2],[x3,y3]]
                    contous_con = {
                        "transcription": 'text',
                        "points": box_,
                        "class": class_
                    }
                    this_img_det_rec.append(contous_con)
                if len(this_img_det_rec) == 0:
                    continue
                want_write_con = datatype + "/" + i + '\t' + json.dumps(this_img_det_rec) + '\n'
                write.write(want_write_con)   
                
make_label()        
make_label(img_dir=r'det_bookspine_text/val',datatype='val') 
                
            

json_data:[{'line_color': [], 'fill_color': [], 'label': 'publish', 'points': [[2291.6426512968296, 42.33573487031697], [2693.371757925072, 41.759365994236276], [2693.5320197044334, 75.04679802955667], [2291.93083573487, 75.47694524495674]], 'group_id': None, 'description': '', 'shape_type': 'polygon', 'flags': {}, 'mask': None}, {'line_color': [], 'fill_color': [], 'label': 'title', 'points': [[1485.3809523809523, 36.587301587301624], [1940.3809523809523, 36.587301587301624], [1940.3809523809523, 86.58730158730162], [1485.3809523809523, 86.58730158730162]], 'group_id': None, 'description': '', 'shape_type': 'polygon', 'flags': {}, 'mask': None}, {'line_color': [], 'fill_color': [], 'label': 'author', 'points': [[1973.8095238095239, 39.07936507936509], [2125.3968253968255, 38.285714285714306], [2125.3968253968255, 80.34920634920638], [1973.8095238095239, 80.34920634920638]], 'group_id': None, 'description': '', 'shape_type': 'polygon', 'flags': {}, 'mask': None}, {'line_color': [], 'fi

In [27]:
[[[274.0, 94.0], [601.0, 94.0], [601.0, 145.0], [274.0, 145.0]], [[1817.0, 53.0], [2021.0, 53.0], [2021.0, 97.0], [1817.0, 97.0]]]

[[[274.0, 94.0], [601.0, 94.0], [601.0, 145.0], [274.0, 145.0]],
 [[1817.0, 53.0], [2021.0, 53.0], [2021.0, 97.0], [1817.0, 97.0]]]